In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

C:\Users\ktser\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [4]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [5]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [8]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421090,36371250.0,39102635.0,10000.0,10000.0,10000.0,36 months,11.99,332.10,B,8 years,...,0.0,1.0,100.0,100.0,0.0,0.0,32950.0,25274.0,9200.0,15850.0
421091,36441262.0,39152692.0,24000.0,24000.0,24000.0,36 months,11.99,797.03,B,10+ years,...,0.0,2.0,56.5,100.0,0.0,0.0,152650.0,8621.0,9000.0,0.0
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0


In [7]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [9]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0
5,68506798.0,73396623.0,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,...,0,0,0,0,0,1,0,0,0,0
6,68566886.0,73456723.0,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,...,0,0,0,0,0,1,0,0,0,0
7,68577849.0,73467703.0,18000.0,18000.0,18000.0,11.99,400.31,112000.0,8.68,0.0,...,0,0,0,0,0,1,0,0,0,0
8,66310712.0,71035433.0,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,0,0,0,0,0,1,0,0,0,0
9,68476807.0,73366655.0,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([ 0.98038517,  0.98000522,  0.98100259,  0.98169124,  0.97499406,
        0.97891237,  0.96953145,  0.98069296,  0.97959959,  0.98050159])

In [9]:
# standardizing the data using sklearn StandardScaler
X = pd.DataFrame(StandardScaler().fit_transform(X))

#Appling PCA to reduce deminsion
pca = PCA(0.90)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents)

In [10]:
principalDf

,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,3.868639,2.668754,-3.086590,2.930553,-1.746278,2.063338,0.973511,-1.129434,1.055081,-3.257188,...,-1.467099,-0.775000,-0.488389,-0.521994,1.276607,2.849805,-0.442854,1.300573,-3.245774,-0.116424
1,-3.267365,0.152531,-2.274563,3.180845,-0.659784,2.040929,1.094666,-1.248240,0.796090,0.831617,...,0.221247,0.117939,0.079369,0.329292,-0.032360,-0.475770,-0.064306,-0.109320,0.001876,-0.016279
2,2.572073,-4.148436,1.679607,3.545030,0.310049,-0.754412,3.918923,-1.893834,-0.709194,0.369997,...,0.118957,0.136175,0.041018,-0.152712,-0.391504,-0.548840,-0.038455,-0.349562,-0.090478,-0.368981
3,4.195051,0.558505,-2.972912,2.319295,-1.116572,-1.666316,3.166701,-1.103758,0.739147,-1.507906,...,0.729616,-0.161247,-0.030986,-0.064265,-0.333551,-0.811134,0.120105,-0.305403,-0.061437,-0.112153
4,-0.335732,4.036574,5.306481,-3.934431,1.072085,2.658298,3.488301,-3.058477,-1.876862,-0.269480,...,1.456464,0.313179,2.754004,0.943539,-2.679644,1.786028,-1.192367,-6.000910,-0.466450,-2.844121
5,2.071519,2.409074,-3.100861,0.471176,-2.154689,2.833451,0.876874,-1.559943,0.861089,1.508695,...,-0.225195,-0.229886,-0.036582,-0.106218,-0.157046,-0.593898,0.015114,-0.033823,-0.080365,0.051628
6,3.669084,-0.449954,-3.838751,-1.994940,0.985067,0.297597,1.998606,-1.596733,0.485018,-0.906126,...,-1.375122,-0.023783,0.022499,0.619137,0.750345,1.016262,-0.194086,0.306933,-0.189107,0.121716
7,3.589621,0.558405,2.354583,1.225766,0.334266,-0.065812,3.605120,-2.088510,0.059704,-1.564559,...,-0.636150,-0.169377,2.473107,-0.203329,-0.973530,0.107880,-0.139298,1.550608,-0.604816,1.616821
8,4.608435,-2.029386,-4.426187,-1.684559,-0.151509,0.133001,2.152564,-1.646857,0.944936,-1.424629,...,-0.832996,-0.035180,-0.083048,0.393668,0.366459,0.303744,-0.120523,0.034964,0.634816,0.083363
9,1.441100,0.166312,1.672910,0.048821,3.941248,3.935807,0.376502,-2.269585,-0.494599,0.262141,...,1.825109,-0.198886,0.619378,-0.264309,-1.069474,0.083667,-0.295415,-1.493864,-0.958850,-1.394025


In [11]:
X = principalDf
Y = y2015['loan_status']

cross_val_score(rfc, X, Y, cv=10)

array([ 0.94557242,  0.94626107,  0.94664102,  0.95003681,  0.93101401,
        0.94006174,  0.94096274,  0.94792087,  0.9380863 ,  0.94696718])